<a href="https://colab.research.google.com/github/isayahc/google_colab/blob/main/OSS_LLM_embedding_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers


In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [3]:
# !pip uninstall llama-cpp-python -y
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install -U llama-cpp-python --no-cache-dir
# !pip install 'llama-cpp-python[server]'
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.7-cp310-cp310-manylinux_2_35_x86_64.whl


In [5]:
!pip install llama-index
!pip install langchain
# !pip install llama-cpp-python
!pip install sentence_transformers

In [6]:
from google.colab import drive

def mount_google_drive() -> None:
    """
    Mount Google Drive to access its files and folders.
    """
    drive.mount('/content/drive')

# Mount Google Drive
mount_google_drive()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os

def navigate_to_directory(directory_path: str) -> None:
    """
    Navigate to the specified directory.

    Args:
        directory_path (str): The path to the directory.
    """
    os.chdir(directory_path)

# Navigate to a specific Google Drive folder
navigate_to_directory('/content/drive/MyDrive/data_data')


In [8]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt


import llama_index.llms.llama_cpp


from langchain.embeddings import HuggingFaceEmbeddings

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"

llm = llama_index.llms.llama_cpp.LlamaCPP( model_kwargs={"n_gpu_layers": 1})


embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# create a service context
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

# load documents
documents = SimpleDirectoryReader(
    "/content/drive/MyDrive/data_data"
).load_data()

# create vector store index
index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# set up query engine
query_engine = index.as_query_engine()


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [9]:
response = query_engine.query("What did the author do growing up?")
print(response)

 Based on the text, there is no mention or hint about what the author did while growing up. The text only discusses the author's research on SWCNTs and their properties.


In [10]:
query_engine = index.as_query_engine()
response = query_engine.query("Who are the authors of this paper?")
print(response)

Llama.generate: prefix-match hit


 Jin-Won Song, Joundong Kim, Yeo-Hwan Yoon, Byung-Sam Choi, Jae-Ho Kim, and Chang-Soo Han.


In [11]:
response = query_engine.query("list all of the sections in the paper")
print(response)

Llama.generate: prefix-match hit


 Based on the provided context information, the following are the sections in the paper:

1. Introduction
2. Experimental Methods
3. Results and Discussion
4. Conclusion
5. References

Note that the section names may vary depending on the specific paper and its formatting style.


In [13]:
response = query_engine.query("summerize this paper")
print(response)


Llama.generate: prefix-match hit



This paper describes a method for direct printing of conductive patterns using SWCNT inkjet technology. The authors investigate the effects of substrate temperature, overwriting time, and linewidth on the properties of the printed lines. They find that the uniformity of the SWCNT network is improved with increasing overwrite times and that the resistance of the lines can be controlled by changing the linewidth and the number of overwritings. The impedance of the printed lines is measured using an impedance analyzer, and it is found to be linearly reduced along with the increase of the overwriting times. The paper demonstrates the potential of SWCNT inkjet technology for fabricating high-performance conductive patterns on various substrates.


In [14]:
response = query_engine.query("what is SWCNT inkjet technology?")
print(response)

Llama.generate: prefix-match hit




SWCNT inkjet technology refers to a method for creating conductive patterns using single-walled carbon nanotubes (SWCNTs) through inkjet printing. This approach allows for the direct printing of patterns, reducing the number of fabrication steps required and eliminating the need for chemical etching or pattern transfer. The technology has the potential to revolutionize the field of electronics and other industries where high-performance conductive materials are needed.

Inkjet printing is a popular method in the conventional printing industry due to its ability to produce fine and arbitrary patterns, non-contact injection, solution saving effects, high repeatability, and scalability. However, most methods of fabricating SWCNT TCF require two fabrication steps to achieve the patterns, which can be time-consuming and costly. In contrast, direct SWCNT printing using an inkjet printing method offers a more efficient and cost-effective solution.

The technology involves dispersing SWCNTs 

In [15]:
response = query_engine.query("describe the method for direct printing of conductive patterns using SWCNT inkjet technology.")
print(response)

Llama.generate: prefix-match hit



The method involves the following steps:
1. Preparing a substrate by plasma treatment or hydrophilic glass to reduce the contact angle of the ink and improve uniformity of drop formation.
2. Dispersing SWCNTs in a solvent, such as DMF, to form an ink.
3. Using an inkjet printer equipped with a piezoelectric actuation module to eject SWCNT droplets onto the substrate.
4. Tuning the jetting conditions, such as the solution concentration, nozzle height, and substrate temperature, to achieve uniform drop formation and prevent nozzle clogging.
5. Controlling the number of overwritings and linewidth to modify the electrical properties of the printed patterns, including resistivity and impedance.
The direct printing approach offers several advantages over traditional methods, including reduced fabrication steps, improved reproducibility, and the ability to control pattern thickness, linewidth, and uniformity. The method has potential applications in large-area electronic devices and transpar

In [16]:
response = query_engine.query("what is DMF?")
print(response)

Llama.generate: prefix-match hit


 Based on the given text, DMF stands for dimethylformamide.


In [17]:
response = query_engine.query("Organize this paper into a newletter")
print(response)

Llama.generate: prefix-match hit



Section 1: Introduction

* Briefly introduce the topic of carbon nanotubes (CNTs) and their potential applications in electronic devices.
* Mention the challenges associated with the uniformity of CNT networks and the need for a controlled printing process.

Section 2: Printing of SWCNTs

* Describe the printing process used in this study, including the type of substrate and the conditions under which the printing was performed.
* Discuss the effect of overwriting time on the uniformity of the printed patterns.

Section 3: Characterization of Inkjet Patterns

* Present the results of the characterization of the printed patterns, including the measurement of electrical resistivity and impedance.
* Discuss the significance of these results in relation to the potential applications of SWCNT-based electronic devices.

Section 4: Conclusion

* Summarize the main findings of the study and their implications for the future development of SWCNT-based electronic devices.
* Mention any potentia

In [18]:
response = query_engine.query("list all of the sources in the paper")
print(response)

Llama.generate: prefix-match hit



1. [1]
2. [2]
3. [3]
4. [5]
5. [6]
6. [7]
7. [8]
8. [9]

Note that the numbers in brackets refer to the sources cited in the paper, and not to any specific order or ranking.


In [19]:
response = query_engine.query("list all of the sources by name")
print(response)

Llama.generate: prefix-match hit



1. [1]
2. [2]
3. [3]
4. [4]
5. [5]
6. [6]
7. [7]
8. [8]
9. [9]
10. [10]
11. [11]
12. [12]
13. [13]
14. [14]
15. [15]
16. [16]
17. [17]
18. [18]


In [20]:
response = query_engine.query("Tell me about the reference section.")
print(response)

Llama.generate: prefix-match hit


 The reference section is a list of sources used in the article. It includes the authors' names, publication dates, titles of articles or journals, volume and page numbers, and URLs.


In [21]:
response = query_engine.query("Tell me about the reference section. An some of the authors")
print(response)

Llama.generate: prefix-match hit


 Sure! Based on the provided context information, here is what I found regarding the reference section:

The article has a total of 18 references cited in the text. These references are listed at the end of the article under the heading "References". The references are numbered and include authors' names, publication titles, and page numbers where applicable. Some of the authors included in the reference section are:

* Song Y I
* Kim G Y
* Choi H K
* Jeong H J
* Kim K K
* Yang C M
* Lim S C
* An K H
* Jung K T
* Lee Y H

These authors have contributed to various research studies related to the topic of the article, which is the inkjet printing of single-walled carbon nanotubes and their electrical characterization.


In [22]:
response = query_engine.query("List as many entities as possible on the paper")
print(response)

Llama.generate: prefix-match hit


 Jin-Won Song, Joundong Kim, Yeo-Hwan Yoon, Byung-Sam Choi, Jae-Ho Kim, Chang-Soo Han

The following are the entities that can be listed from the given context information:

1. Authors of the paper: Jin-Won Song, Joundong Kim, Yeo-Hwan Yoon, Byung-Sam Choi, Jae-Ho Kim, Chang-Soo Han
2. Research centers: Nano-Mechanical Systems Research Center, Korea Institute of Machinery and Materials, Ajou University
3. Departments: Molecular Science and Technology, Department of Molecular Science and Technology
4. Universities: Ajou University
5. Countries: Korea
6. Cities: Daejeon, Suwon
7. Substrates: glass, polymer substrate
8. Electrical properties: resistance, conductivity, impedance, ohmic characteristic
9. Measurement tools: probe station (Cascade Special DC package), precision multi-meter(Keithley 2400)
10. Nanosc


In [23]:
text = """The following are the entities that can be listed from the given context information:

1. Authors of the paper: Jin-Won Song, Joundong Kim, Yeo-Hwan Yoon, Byung-Sam Choi, Jae-Ho Kim, Chang-Soo Han
2. Research centers: Nano-Mechanical Systems Research Center, Korea Institute of Machinery and Materials, Ajou University
3. Departments: Molecular Science and Technology, Department of Molecular Science and Technology
4. Universities: Ajou University
5. Countries: Korea
6. Cities: Daejeon, Suwon
7. Substrates: glass, polymer substrate
8. Electrical properties: resistance, conductivity, impedance, ohmic characteristic
9. Measurement tools: probe station (Cascade Special DC package), precision multi-meter(Keithley 2400)
Continue where you left off"""


In [24]:
response = query_engine.query(text)
print(response)

Llama.generate: prefix-match hit


1, 2, 3, 4, 5, 6, 7, 8, 9.


In [ ]:
response = query_engine.query("categorize the entites in this paper")
print(response)

Llama.generate: prefix-match hit
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-25-71357834ed58>", line 1, in <cell line: 1>
    response = query_engine.query("categorize the entites in this paper")
  File "/usr/local/lib/python3.10/dist-packages/llama_index/indices/query/base.py", line 23, in query
    response = self._query(str_or_query_bundle)
  File "/usr/local/lib/python3.10/dist-packages/llama_index/query_engine/retriever_query_engine.py", line 178, in _query
    response = self._response_synthesizer.synthesize(
  File "/usr/local/lib/python3.10/dist-packages/llama_index/response_synthesizers/base.py", line 128, in synthesize
    response_str = self.get_response(
  File "/usr/local/lib/python3.10/dist-packages/llama_index/response_synthesizers/compact_and_refine.py", line 34, in get_response
    response = super().get_response(
  File "/usr/loc